In [3]:
import os
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import json
from tqdm import tqdm

print("PyTorch Version:", torch.__version__)

PyTorch Version: 2.9.1+cpu


In [4]:
class SimpsonsCNN(nn.Module):
    """Custom CNN for Simpsons character classification"""
    
    def __init__(self, num_classes=42):
        super(SimpsonsCNN, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25)
        )
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25)
        )
        
        # Fully Connected Layers
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 8 * 8, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.fc(x)
        return x


In [5]:
def load_model(model_path, device):
    """Load the trained model from checkpoint"""
    
    # Load checkpoint
    checkpoint = torch.load(model_path, map_location=device)
    
    # Get class information
    idx_to_class = checkpoint['idx_to_class']
    num_classes = len(idx_to_class)
    
    # Initialize model
    model = SimpsonsCNN(num_classes=num_classes)
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    model.eval()
    
    print(f"Model loaded from: {model_path}")
    print(f"Model trained for {num_classes} classes")
    print(f"Best validation loss: {checkpoint['val_loss']:.4f}")
    print(f"Best validation accuracy: {checkpoint['val_acc']:.2f}%")
    print(f"Best validation F1: {checkpoint['val_f1']:.4f}")
    
    return model, idx_to_class

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\nUsing device: {device}")

# Load model
model_path = r"C:\Users\User\Desktop\springfield_identity\models\simpsons_cnn.pth"
model, idx_to_class = load_model(model_path, device)


Using device: cpu
Model loaded from: C:\Users\User\Desktop\springfield_identity\models\simpsons_cnn.pth
Model trained for 42 classes
Best validation loss: 1.1028
Best validation accuracy: 65.88%
Best validation F1: 0.5311


In [6]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225])
])

In [7]:
def infer(data_dir, model_path):
    """
    Run inference on all images in data_dir and save results to results.json
    
    Args:
        data_dir: Path to directory containing test images
        model_path: Path to saved model checkpoint
    """
    
    # Setup device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load model
    model, idx_to_class = load_model(model_path, device)
    
    # Get all jpg images in data_dir
    image_files = [f for f in os.listdir(data_dir) if f.endswith('.jpg')]
    
    if len(image_files) == 0:
        print(f"No .jpg images found in {data_dir}")
        return
    
    print(f"\nFound {len(image_files)} images to process.")
    
    # Dictionary to store results
    results = {}
    
    # Process each image
    with torch.no_grad():
        for img_file in tqdm(image_files, desc="Processing images"):
            img_path = os.path.join(data_dir, img_file)
            
            try:
                # Load and preprocess image
                image = Image.open(img_path).convert('RGB')
                image_tensor = transform(image).unsqueeze(0).to(device)
                
                # Get prediction
                output = model(image_tensor)
                _, predicted = torch.max(output, 1)
                predicted_class = idx_to_class[str(predicted.item())]
                
                # Store result
                results[img_file] = predicted_class
                
            except Exception as e:
                print(f"Error processing {img_file}: {e}")
                continue
    
    # Save results to JSON
    output_path = "results.json"
    with open(output_path, 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"\nInference complete!")
    print(f"Processed {len(results)} images")
    print(f"Results saved to: {output_path}")
    
    return results
